In [1]:
import time

import datetime
import pytz
import calendar
import itertools
import pandas as pd

import datetime
import sys

import numpy as np
import os
import pandas as pd
import pytz

sys.path.insert(0, '../')
sys.path.insert(0, '../Optimizers')

# import utils
# from utils import plotly_figure

import itertools
import xbos_services_getter as xsg

from Optimizers.MPC import MPC
from Optimizers.MPC import Node
from DataManager.DataManager import DataManager

In [4]:
class Tstat:
    def __init__(self, building, zone, temperature):
        self.temperature = temperature
        self.indoor_temperature_prediction_stub =  xsg.get_indoor_temperature_prediction_stub()
        self.error = {}
        for action in [xsg.NO_ACTION, xsg.HEATING_ACTION, xsg.COOLING_ACTION]:
            mean, var = xsg.get_indoor_temperature_prediction_error(self.indoor_temperature_prediction_stub,
                                                                   building,
                                                                   zone, 
                                                                   action)
            self.error[action] = {"mean": mean, "var": var}
        
    def next_temperature(self, action):
        self.temperature +=  1 * (action==1) - 1*(action==2) + np.random.normal(self.error[action]["mean"],
                                                                                  self.error[action]["var"])
        return self.temperature
    

In [26]:
class SimulationMPC(DataManager):
    
    def __init__(self, building, zones, lambda_val, start, end, forecasting_horizon, window, tstats, non_contrallable_data=None):
        
        assert xsg.get_window_in_sec(forecasting_horizon) % xsg.get_window_in_sec(window) == 0
        
        self.building = building
        self.zones = zones
        self.lambda_val = lambda_val
        
        self.forecasting_horizon = forecasting_horizon
        self.delta_forecasting_horizon = datetime.timedelta(seconds=xsg.get_window_in_sec(forecasting_horizon))
        
        self.delta_window = datetime.timedelta(seconds=xsg.get_window_in_sec(window))
        
        # Simulation end is when current_time reaches end and end will become the end of our data. 
        self.simulation_end = end 
        end += self.delta_forecasting_horizon
        
        super().__init__(building, zones, start, end, window, non_contrallable_data)
        
        
        
        self.tstats = tstats# dictionary of simulator object with key zone. has functions: current_temperature, next_temperature(action)
                
        self.current_time = start 
        self.current_time_step = 0
        
        self.actions = {iter_zone: [] for iter_zone in self.zones} # {zone: [ints]}
        self.temperatures = {iter_zone: [self.tstats[iter_zone].temperature] for iter_zone in self.zones} # {zone: [floats]}
        
        
    def step(self):
        
        # call
        start_mpc = self.current_time
        end_mpc = self.current_time + self.delta_forecasting_horizon
        non_controllable_data = {
            "comfortband": {iter_zone: self.comfortband[iter_zone].loc[start_mpc:end_mpc] for iter_zone in self.zones},
            "do_not_exceed": {iter_zone: self.do_not_exceed[iter_zone].loc[start_mpc:end_mpc] for iter_zone in self.zones},
            "occupancy": {iter_zone: self.occupancy[iter_zone].loc[start_mpc:end_mpc] for iter_zone in self.zones},
            "outdoor_temperature": self.outdoor_temperature.loc[start_mpc:end_mpc]
        }

        op = MPC(self.building, self.zones, start_mpc, end_mpc, self.window, self.lambda_val, non_controllable_data=non_controllable_data,
                 debug=False)
        
        root = Node({iter_zone: tstats[iter_zone].temperature for iter_zone in self.zones}, 0)
        
        root = op.shortest_path(root)
        best_action = op.g.node[root]["best_action"]        
        
        
        # given the actions, update simulation of temperature. 
        # increment time
        
        self.current_time += self.delta_window
        self.current_time_step += 1
        
        for iter_zone in self.zones:
            # advances temperature and saves it
            self.temperatures[iter_zone].append(self.tstats[iter_zone].next_temperature(best_action[iter_zone]))
            self.actions[iter_zone].append(best_action[iter_zone])
        
        return root

    def run(self):
        while self.current_time < self.simulation_end:
            print(self.current_time_step)
            self.step()
        


In [31]:
forecasting_horizon = "4h"

end = datetime.datetime.utcnow().replace(tzinfo=pytz.utc) - datetime.timedelta(seconds=xsg.get_window_in_sec(forecasting_horizon))
end = end.replace(microsecond=0)
start =  end - datetime.timedelta(hours=6)

print(start)
print(start.timestamp())
building = "avenal-animal-shelter"
zones = ["hvac_zone_shelter_corridor"]
window = "15m"
lambda_val = 0.995
tstats = {iter_zone: Tstat(building, iter_zone, 75) for iter_zone in zones}

2019-04-29 17:02:27+00:00
1556557347.0


In [32]:
simulation = SimulationMPC(building, zones, lambda_val, start, end, forecasting_horizon, window, tstats)

In [30]:
t = time.time()
simulation.step()
print(time.time() - t)
# print(simulation.temperatures)
# print(simulation.actions)


KeyboardInterrupt: 

In [33]:
a = time.time()
simulation.run()
b = time.time()
print( b -a)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
43.33914303779602


In [36]:
print(simulation.current_time)
print(simulation.end)
print(simulation.current_time  simulation.end)

2019-04-29 23:02:27+00:00
2019-04-30 03:02:27+00:00
False


In [37]:
print(simulation.temperatures)
print(simulation.actions)

{'hvac_zone_shelter_corridor': [75, 75.00791627560486, 74.98398963570554, 75.00190642278805, 75.01340860913935, 75.0207883625172, 75.03388366764226, 75.04884500915144, 75.0200034771109, 75.02531380498823, 75.0101123219211, 74.99872276260515, 74.99089478158713, 74.99505538945354, 74.98943657058025, 74.97508563567006, 74.97575747983373, 74.97825249692207, 74.95885275934913, 74.9546498576091, 74.9745250505232, 74.93827939993965, 74.93929251498903, 74.93810156103561, 74.94459185057266]}
{'hvac_zone_shelter_corridor': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [ ]:
tstat = Tstat("avenal-animal-shelter", "hvac_zone_shelter_corridor", 70)

In [ ]:
tstat.error

In [ ]:
tstat.next_temperature(1)